# **Task 1 - Initial System (Arabic dataset)**

## a) Data Loading & Inspection

In [1]:
# Import helper functions from `tools/reader/CT20_AR_reader.py`
from tools.readers.CT20_AR_reader import readTweets, readLabels, readTopics

# Import other modules
import googletrans
import emoji

In [2]:
# Read in tweets, labels & topics using helper functions
tweets = readTweets()
labels = readLabels()
topics = readTopics()

In [3]:
print(type(tweets))
print('Number of tweets:', len(tweets), '\n')
print(tweets[0])

<class 'list'>
Number of tweets: 1500 

{'created_at': 'Mon Jan 27 00:09:17 +0000 2020', 'id': 1221585936095555584, 'id_str': '1221585936095555584', 'full_text': 'بين ثورة لبنان ال١٠٠ يوم وبين ٤٠ العراق \nتتر ورُعاع ايران ما زالت رائحة البارود والنار تفوح منهم ، من حرق الخيم وإطلاق الرصاص على المتظاهرين \nوأكتافهم تعبت من بناء حواجز بينهم وبين المواطنين لحماية مملكتهم الهشة \n\nرُعاع القرن الحالي \n#ايران #العراق #لبنان \n#لبنان_النا_مش_الكن', 'truncated': False, 'display_text_range': [0, 276], 'entities': {'hashtags': [{'text': 'ايران', 'indices': [234, 240]}, {'text': 'العراق', 'indices': [241, 248]}, {'text': 'لبنان', 'indices': [249, 255]}, {'text': 'لبنان_النا_مش_الكن', 'indices': [257, 276]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'ar', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_u

In [4]:
labels.head()

,topicID,tweetID,label
0,CT20-AR-05,1221585936095555584,0
1,CT20-AR-05,1221587916750753793,0
2,CT20-AR-05,1221602993918894081,0
3,CT20-AR-05,1221653116036304896,0
4,CT20-AR-05,1221663019345874944,1


In [5]:
topics.head()

,topicID,title,description
0,CT20-AR-05,الاحتجاجات في لبنان,"يتحدث الموضوع عن الاحتجاجات السياسية في لبنان ويغطي مواضيع فرعية منها أسباب الاحتجاجات، رفض الحكومة الجديدة، فضيحة النفايات، أموال بروكسل المنهوبة، قضية اختلاسات الحريري, انقسام القوة السياسية الداعمة لدياب,واقرار موازنة 2020"
1,CT20-AR-10,وسيم يوسف,وسيم يوسف هو داعية إماراتي من أصل أردني غالباً ما يثير الجدل. يغطي الموضوع التغريدات التي قام بها وسيم يوسف وتلك التي تذكره او تهاجمه أو تذكر مواقفه السياسية بخصوص الإمارات وحكامها ودول المنطقة بالإضافة إلى آرائه الدينية مثل مهاجمة البخاري.
2,CT20-AR-19,تدخل تركيا في سوريا,بعد استمرار الحرب في سوريا ما يُقارِب التسع سنين بعد اشتعال الثورة عام 2011، قررت تركيا الدخول بهدف معلن وهو حماية المدنيين وردع القوات السورية والأجنبية عن توتير الأوضاع وقتل وتشريد المدنيين، وهو ما أثار الرأي العام في العالم. يتحدث هذا الموضوع عن تطورات التدخل التركي العسكري في سوريا على جميع الأصعدة


## b) Data Preprocessing

**NOTE -** this cell takes ~10 minutes to run due to the number of requests to Google Translate.

In [11]:
# Create new DF to contain tweet text (English & Arabic), ID, topic & label
tweets_df = labels
tweets_df['full_text_ar'] = 0
tweets_df['full_text_en'] = 0
translate_urls = ["translate.google.com", "translate.google.co.kr",
                  "translate.google.at", "translate.google.de",
                  "translate.google.ru", "translate.google.ch",
                  "translate.google.fr", "translate.google.es"]

# Function to remove emojis as translator doesn't handle these well
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

for tweet in tweets:
    # For each tweet ID, find row with that ID in DataFrame and add Arabic text to that row
    tweet_idx = tweets_df[tweets_df['tweetID'] == tweet['id']].index[0]
    emojiless_text = remove_emoji(tweet['full_text'])
    tweets_df.iloc[tweet_idx, tweets_df.columns.get_loc('full_text_ar')] = emojiless_text
    
    # Use googletrans package to translate text from Arabic to English & store in DataFrame
    translator = googletrans.Translator(service_urls=translate_urls)
    try: # Try translation, if it fails, insert '0' into the DataFrame
        translation = translator.translate(emojiless_text, dest='en', src='ar').text
    except:
        translation = 0
    tweets_df.iloc[tweet_idx, tweets_df.columns.get_loc('full_text_en')] = translation

In [20]:
print(len(tweets_df[tweets_df['full_text_en'] != 0]), 'texts translated out of 1500')
tweets_df = tweets_df[tweets_df['full_text_en'] != 0] # Remove the one tweet which failed to translate
tweets_df.to_csv('translated_tweets_t1.csv') # Save translated tweets as .csv
tweets_df.head(1500)

1499 texts translated out of 1500


,topicID,tweetID,label,full_text_ar,full_text_en
0,CT20-AR-05,1221585936095555584,0,بين ثورة لبنان ال١٠٠ يوم وبين ٤٠ العراق \nتتر ورُعاع ايران ما زالت رائحة البارود والنار تفوح منهم ، من حرق الخيم وإطلاق الرصاص على المتظاهرين \nوأكتافهم تعبت من بناء حواجز بينهم وبين المواطنين لحماية مملكتهم الهشة \n\nرُعاع القرن الحالي \n#ايران #العراق #لبنان \n#لبنان_النا_مش_الكن,"Between Lebanon and the 100-day revolution between 40 and Iraq\nTatar and rabble Iran is the smell of gunpowder and fire still smelt them, from the burning tents and shooting at demonstrators\nAnd their shoulders are tired of building including barriers between citizens and to protect the fragile kingdom\n\nRiffraff current century\nIran # # # Iraq, Lebanon\n# Of Banan_na_mh_alkn"
1,CT20-AR-05,1221587916750753793,0,#لبنان_النا_مش_الكن\nأرملة فقيرة\nعليهاإيقاف خدمات\nخلونانفرحهاوعيالها\nليس لهم بعد الله سبحانه إلاأنتم\nبيت أجاروعايشه ع الضمان\nوفاتورة الكهرباء أهم وأقل شي سداد نص الفاتورة\nوالفصل بعديوم\nأسأل الله يفرج همهم\nوالله مبلغهم مو معجزة\nالتنفيذ؛2030633080(77ألف)\nالكهرباء؛10097517557(7آلاف) https://t.co/1kzQxf2XeB,# Of Banan_na_mh_alkn\nPoor widow\nAllehaaaagaf Services\nKhllonanfrhaauaaalha\nNot for them after God Almighty Alaontm\nHouse Oquaaish p Guarantee\nThe electricity bill the most important and less Shi to pay the bill text\nThe chapter After a day\nI ask God released hum\nGod Mpelghm Mo miracle\nImplementation; 2030633080 (77 thousand)\nElectricity; 10097517557 (7 thousand) https://t.co/1kzQxf2XeB
2,CT20-AR-05,1221602993918894081,0,#جدار_العار\nالحيتان الحيطان..عمروا بينن و بين الشعب..حيطان,# Jaddar_ar\nWhales Alehitan..amra Bann and between Alhab..hitan
3,CT20-AR-05,1221653116036304896,0,هل يحمي #جدار_العار السلطة السياسية الفاسدة من شعبها الثائر ضدها؟ https://t.co/HYQnmpI07L,Does it protect # Jaddar_ar corrupt political power of its people rebellious against it? https://t.co/HYQnmpI07L
4,CT20-AR-05,1221663019345874944,1,موازنة شو؟!\n١- ارقام غير صحيحة.\n٢- حكومة لم تنل الثقة بعد، يعني غير مسؤولة عن ارقام وضعتها الحكومة السابقة.\nانّو جلسة استفزاز للشعب؟!\nاذا بدكن الناس تعطيكم فرصة، بالقليلة كونوا منطقيين!\n#لبنان_ينتفض,"Balancing Shu ?!\n1. The figures are incorrect.\n2. The Government has not received the confidence yet, I mean is not responsible for the figures set by the previous government.\nLlano session provocation to the people ?!\nIf people give you a chance Bdkin, inconsiderable Be logical!\n# Of Banan_antvd"
...,...,...,...,...,...
1495,CT20-AR-19,1232376139148152832,0,في برنامج الاتجاه المعاكس الليلة:\nهل ينفجر الوضع بين تركيا وروسيا في شمال سوريا؟ https://t.co/8k4gceEg6t,In the program opposite direction tonight:\nDo you explode the situation between Turkey and Russia in the north of Syria? https://t.co/8k4gceEg6t
1496,CT20-AR-19,1232381112204193792,0,#الاتجاه_المعاكس - أليس من حق #تركيا أن تحمي أمنها القومي في شمال #سوريا؟ أم إنها تورطت في المستنقع السوري؟ https://t.co/KOh90wOZC5,# Alatjah_alamaaks - Is not it right # Turkey to protect its national security in northern Syria #? Or is it involved in the Syrian quagmire? https://t.co/KOh90wOZC5
1497,CT20-AR-19,1232395009325965317,0,ما أهمية جبل الزاوية في إدلب.. وما الذي تعنيه محاولة روسيا والأسد التقدم عليه؟ وكيف سيكون موقف تركيا؟.. خبراء يجيبون\n\nhttps://t.co/qziOehgC1w,What is the importance of Mount corner in Idlib .. What do you mean trying to Russia and Assad progress on it? How will Turkey's position? .. Experts answer\n\nhttps://t.co/qziOehgC1w
1498,CT20-AR-19,1232397768573976579,0,يرى البلدان (تركيا وروسيا)أن منطقة البحر الأسود هي مجال نفوذهما الطبيعي، وهما يواصلان الصراع في ليبيا ويشتبكان بمعارك مباشرة في إدلب وسيتم ضمان المصالح طويلة الأجل لكل من الولايات المتحدة وتركيا بأفضل شكل، من خلال الاستمرار في العمل معا لتحسين التعاون ضد روسيا ثنائيا وفي الناتو.,"See countries (Turkey and Russia) that Sea region Black is the area of ​​natural influence, and are continuing to struggle in Libya and Eshetbakan battles directly in Idlib and will be term guarantee long interests of bo